In [1]:
# | export

from dreamai.core import *
from dreamai.vision import *
from dreamai.imports import *
from langchain_ray.imports import *
from langchain_ray.utils import *
from langchain_ray.chains import *
import pdfplumber
from pypdf import PdfReader

In [2]:
Path("/home/hamza/dev/HF/langchain_ray/resumes_5").exists()

True

In [2]:
a = ["docs"]
b = [["d1", "d2", "d3"]]
{k: v for k, v in zip(a, b)}

{'docs': ['d1', 'd2', 'd3']}

In [ ]:
#| hide

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
def empty_row(row):
    return len([c for c in row if c not in ["", None]]) == 0


def full_row(row, cols):
    return len([c for c in row if c not in ["", None]]) == len(cols)


def is_empty(x):
    return x in ["", None]


def find_empty_id(table):
    for i, row in enumerate(table):
        if empty_row(row):
            return i
    return -1


def page_to_order(
    page,
    table_settings={
        "horizontal_strategy": "text",
        "vertical_strategy": "lines",
        "intersection_x_tolerance": 5,
        "snap_y_tolerance": 5,
    },
):
    table = page.extract_table(table_settings=table_settings)
    cols_id = find_empty_id(table) + 1
    cols = table[cols_id]
    cols_dict = {c: [] for c in cols}
    cols_id += 2
    comments = []
    for info_id, row in enumerate(table[cols_id:]):
        if empty_row(row) or full_row(row, cols):
            break
        elif is_empty(row[0]) and not is_empty(row[1]):
            comments.append(row[1])
    info_id += cols_id
    lines = []
    for row in table[info_id:]:
        if empty_row(row) and len(lines) > 0:
            cols_dict[cols[1]].append(", ".join(lines))
            lines = []
        else:
            if not is_empty(row[0]):
                cols_dict[cols[0]].append(row[0])
            if not is_empty(row[1]):
                lines.append(row[1])
            for r, c in zip(row[2:], cols[2:]):
                if not is_empty(r):
                    cols_dict[c].append(r)
    if len(lines) > 0:
        cols_dict[cols[1]].append(", ".join(lines))
    cd0 = cols_dict[cols[0]]
    cd_1 = cols_dict[cols[-1]]
    cd_2 = cols_dict[cols[-2]]
    if len(cd_1) > len(cd0) and len(cd_2) > len(cd0):
        total = float(cd_1[-1].replace("$", "").replace(",", "").strip())
        cols_dict[cols[-1]] = cd_1[: len(cd0)]
        cols_dict[cols[-2]] = cd_2[: len(cd0)]
    else:
        total = 0
    return pd.DataFrame(cols_dict), ", ".join(comments), total


def pdf_to_order(pdf):
    pdf = pdfplumber.open(pdf)
    dfs = []
    comments_ = []
    totals = []
    for page in pdf.pages:
        df, comments, total = page_to_order(page)
        dfs.append(df)
        comments_.append(comments)
        totals.append(total)
    order = pd.concat(dfs).reset_index(drop=True)
    comments = comments_[0]
    total = totals[-1]
    return order, comments, total


def page_to_extra_info(page):
    text = page.extract_text().splitlines()
    ik2 = ["Purchase Order No.", "Incoterms", "Payment Terms", "Method of Shipping"]
    ik3 = ["Target Ship Date", "Ship To", "Currency"]
    info_dict = {}
    for k2 in ik2:
        for t in text:
            if t.startswith(k2):
                t = t.replace(k2, "").strip()
                for k3 in ik3:
                    idx = t.find(k3)
                    if idx != -1:
                        info_dict[k2] = t[:idx]
                        info_dict[k3] = t[idx:].replace(k3, "").replace(":", "").strip()
                        break
                    else:
                        info_dict[k2] = t
                break
    return info_dict


def pdf_to_info(pdf):
    page = pdfplumber.open(pdf).pages[0]
    tables = page.extract_tables()
    info_table = tables[0][1][0].split("\n")
    address_table = tables[1][1][0].split("\n")
    info_keys = [
        "Order Number",
        "Revision",
        "Document Date",
        "Customer No.",
        "Quotation",
        "Questions/Contact",
        "Email",
        "Requested by",
    ]
    info_dict = {k: t.replace(k, "").strip() for k, t in zip(info_keys, info_table)}
    address_dict = {
        "Address": ", ".join(address_table[:-2]),
        "Tel": address_table[-2].replace("Tel#", "").strip(),
        "Fax": address_table[-1].replace("Fax#", "").strip(),
    }
    info_dict.update(address_dict)
    extra_info = page_to_extra_info(page)
    info_dict.update(extra_info)
    info_dict = {k: [v] for k, v in info_dict.items()}
    return pd.DataFrame(info_dict)


def pdf_to_dfs(pdf):
    info_dfs = []
    order_dfs = []
    for pdf in resolve_data_path(pdf):
        if Path(pdf).suffix == ".pdf":
            order_info = pdf_to_info(pdf)
            order, comments, total = pdf_to_order(pdf)
            order_info = pdf_to_info(pdf)
            order["Order Number"] = order_info["Order Number"][0]
            order_info["Comments"] = comments
            order_info["Total"] = total
            info_dfs.append(pd.DataFrame(order_info))
            order_dfs.append(
                pd.DataFrame(order, columns=["Order Number"] + list(order.keys())[:-1])
            )
    info_df = pd.concat(info_dfs).reset_index(drop=True)
    order_df = pd.concat(order_dfs).reset_index(drop=True)
    return info_df, order_df


def load_csv(path):
    loader = CSVLoader(file_path=path)
    return loader.load()


def load_csv_chain(input_variables=["csv"], output_variables=["csv_data"], verbose=False):
    return transform_chain(
        load_csv,
        vars_kwargs_mapping={input_variables[0]: "path"},
        input_variables=input_variables,
        output_variables=output_variables,
        verbose=verbose,
    )


def pdf_to_dfs_chain(
    input_variables=["pdf"], output_variables=["info_df", "order_df"], verbose=False
):
    return transform_chain(
        pdf_to_dfs,
        vars_kwargs_mapping={input_variables[0]: "pdf"},
        input_variables=input_variables,
        output_variables=output_variables,
        verbose=verbose,
    )

In [ ]:
pdfs = ["/media/hamza/data2/wt1.pdf"]
pdf_chain = pdf_to_dfs_chain(
    input_variables=["pdf"], output_variables=["info_df", "order_df"], verbose=False
)
pdf_dfs = pdf_chain(dict(pdf=pdfs))
info_df = pdf_dfs["info_df"]
order_df = pdf_dfs["order_df"]

In [ ]:
info_df

In [ ]:
order_df

In [ ]:
with open('/media/hamza/data2/wt1_info.json', 'w') as f:
    json.dump(info_df.to_dict('records'), f, indent=4)

In [ ]:
with open('/media/hamza/data2/wt1_order.json', 'w') as f:
    json.dump(order_df.to_dict('records'), f, indent=4)

In [ ]:
llm = HuggingFacePipeline(
    pipeline=pipeline(
        "text2text-generation",
        model="google/flan-t5-large",
        device_map=default_device(),
        max_new_tokens=256,
    )
)

In [ ]:
csv_chain = load_csv_chain()

In [ ]:
qty_temp = """This is the context: {csv_data}
What was the quantity of {input}?
"""

qty_prompt = PromptTemplate.from_template(qty_temp)
llm_chain = LLMChain(prompt=qty_prompt, llm=llm)
qty_chain = SequentialChain(chains=[csv_chain, llm_chain], input_variables=["csv", "input"])

input = "25781"
qty_chain.run(csv=df_path, input=input)